In [423]:
import pandas as pd
import numpy as np
import math
from sklearn import preprocessing
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances
import warnings
warnings.filterwarnings('ignore')
#MIGHT HAVE TO ENCODE THE BINNING OF THE LENGTHS SO THIS CAN BE A 
#PARAMETER FOR A COLD START

In [424]:
US_trails = pd.read_csv('../data/US_trails_engineered.csv')

In [425]:
US_trails.columns

Index(['name', 'location', 'difficulty', 'length', 'ascent', 'descent',
       'stars', 'latitude', 'longitude', 'summary', 'url',
       'difficulty_encoded', 'category', 'type_Featured Ride', 'type_Trail',
       'city/town', 'state', 'length_range'],
      dtype='object')

In [426]:
US_trails.shape

(20691, 18)

In [427]:
US_trails.dtypes

name                   object
location               object
difficulty             object
length                float64
ascent                  int64
descent                 int64
stars                 float64
latitude              float64
longitude             float64
summary                object
url                    object
difficulty_encoded      int64
category               object
type_Featured Ride      int64
type_Trail              int64
city/town              object
state                  object
length_range           object
dtype: object

In [428]:
#I don't need the latitude and longitude columns since I'll be filtering on those
columns_to_scale = list(US_trails.select_dtypes(exclude=['object']).columns.difference(['latitude', 'longitude', 'stars']))
columns_to_scale

['ascent',
 'descent',
 'difficulty_encoded',
 'length',
 'type_Featured Ride',
 'type_Trail']

In [429]:
#this scales each column individually, BUT REALLY WHAT IS IT DOING?
for col in columns_to_scale:
    US_trails[col + "_scaled"] = preprocessing.scale(US_trails[col])

In [430]:
'''
#THIS DOES THE SAME AS ABOVE, EXCEPT THAT IT REWRITES ALL THE QUANTITATIVE COLUMNS WITH THEIR SCALED VALUES
scaler = preprocessing.StandardScaler()
scaler.fit(US_trails[quantitative_columns])
US_trails[quantitative_columns] = scaler.transform(US_trails[quantitative_columns])
US_trails.head()
'''

'\n#THIS DOES THE SAME AS ABOVE, EXCEPT THAT IT REWRITES ALL THE QUANTITATIVE COLUMNS WITH THEIR SCALED VALUES\nscaler = preprocessing.StandardScaler()\nscaler.fit(US_trails[quantitative_columns])\nUS_trails[quantitative_columns] = scaler.transform(US_trails[quantitative_columns])\nUS_trails.head()\n'

In [431]:
US_trails.columns

Index(['name', 'location', 'difficulty', 'length', 'ascent', 'descent',
       'stars', 'latitude', 'longitude', 'summary', 'url',
       'difficulty_encoded', 'category', 'type_Featured Ride', 'type_Trail',
       'city/town', 'state', 'length_range', 'ascent_scaled', 'descent_scaled',
       'difficulty_encoded_scaled', 'length_scaled',
       'type_Featured Ride_scaled', 'type_Trail_scaled'],
      dtype='object')

In [432]:
scaled_cols = [col for col in US_trails.columns if 'scaled' in col]
scaled_cols
#US_trails_scaled = US_trails.filter(regex='scaled')
#US_trails_scaled.head()

['ascent_scaled',
 'descent_scaled',
 'difficulty_encoded_scaled',
 'length_scaled',
 'type_Featured Ride_scaled',
 'type_Trail_scaled']

In [433]:
Big_X = US_trails[scaled_cols].values
Big_X

array([[ 2.948716  , -2.79569706,  2.40444594,  2.09771539,  2.6075075 ,
        -2.6075075 ],
       [-0.09257181,  0.12246069,  1.6195909 , -0.07002184,  2.6075075 ,
        -2.6075075 ],
       [-0.1825315 ,  0.20147031, -0.7349742 , -0.07002184,  2.6075075 ,
        -2.6075075 ],
       ..., 
       [ 0.86338537, -0.22996377, -0.7349742 ,  1.80477792, -0.38350801,
         0.38350801],
       [-0.12617121, -0.05635054, -0.7349742 ,  0.53928809, -0.38350801,
         0.38350801],
       [ 2.32333359, -1.441098  ,  0.04988084,  1.53527546, -0.38350801,
         0.38350801]])

In [434]:
#this code is from the web app
def cos_sim_recs(index, desired_state=None, desired_city_town=None):
    trail = X[index].reshape(1,-1)
    cs = cosine_similarity(trail, X)
    rec_index = np.argsort(cs)[0][::-1][1:]
    ordered_df = df.loc[rec_index]
    if desired_state is not None:
        ordered_df = ordered_df[ordered_df['state']== desired_state]
    if desired_city_town is not None:
        ordered_df = ordered_df[ordered_df['city/town']== desired_city_town]
    #rec_df = ordered_df.head(n)
    rec_df = ordered_df.head(20)
    rec_df = rec_df.reset_index(drop=True)
    rec_df.index = rec_df.index+1 #this makes it so that there's no index 0 shown
    orig_row = df.loc[[index]].rename(lambda x: 'original')
    total = pd.concat([orig_row, rec_df])
    columns_to_output = ['name', 'location', 'difficulty', 'stars', 'length', 'ascent', 'descent',
       'category', 'summary', 'url']
    output = total[columns_to_output]
    return output

In [435]:
#this function will take in a state you ridden in, a trail you rode on, and the state you want to ride in and recommend
def cos_sim_recommendations(trail_name, state_name, scaled_features, desired_state = None, desired_city_town=None):
    index = US_trails.index[(US_trails['name'] == trail_name) & (US_trails['state']==state_name)][0]
    #index is the index of the given trail in the main df based on it's name and state
    trail = scaled_features[index].reshape(1,-1)
    print('The scaled features of the original trail are:{0}'.format(trail))
    #trail is the scaled features of the given trail reshaped into a 1 row array
    cs = cosine_similarity(trail, scaled_features)
    #cs is the array of cosine similarity measurements of the trail compared to the scaled_features of the other trails
    rec_index = np.argsort(cs)[0][::-1][1:]
    #rec_index is an array of the indices of the cosine similarity measurements ordered by greatest to least, leaving out the given trail
    print('The sorted cosine similarity measurements are:{0}'.format(np.sort(cs[0])[::-1])) #.sort(reverse = True)))
    print('The trail most similiar to the original trail has features: {0}'.format(scaled_features[rec_index][0]))
    ordered_df = US_trails.loc[rec_index]
    print('The trail most similar to the original trail is: {0}'.format(ordered_df.iloc[0]['name']))
    #ordered_df gives the trails of US_trails ordered by the trails most similar to least similar
    if desired_state is not None:
        ordered_df = ordered_df[ordered_df['state']== desired_state]
    #if a desired state to ride in is given, the dataframe is returned with trails only in that state
    if desired_city_town is not None:
        ordered_df = ordered_df[ordered_df['city/town']== desired_city_town]
    #if a desired location to ride in is given, the dataframe is returned with trails only in that location
    rec_df = ordered_df.head(20)
    #rec_df is a dataframe with the number of desired trails to be recommended
    orig_row = US_trails.loc[[index]].rename(lambda x: 'original')
    #orig_row renames the index of the original trail given as 'original'
    total = pd.concat([orig_row, rec_df])
    #this combines the original row labeled as original row and the n recommended trails that are most similar to the original
    return total
    

In [436]:
#this function will take in a state you ridden in, a trail you rode on, and the state you want to ride in and recommend
def euclidean_dist_recommendations(trail_name, state_name, scaled_features, desired_state = None, desired_city_town= None):
    index = US_trails.index[(US_trails['name'] == trail_name) & (US_trails['state']==state_name)][0]
    #index is the index of the given trail in the main df based on it's name and state
    trail = scaled_features[index].reshape(1,-1)
    print('The scaled features of the original trail are:{0}'.format(trail))
    #trail is the scaled features of the given trail reshaped into a 1 row array
    ed = euclidean_distances(trail, scaled_features)
    #ed is the array of euclidean distance measurements of the trail compared to the scaled_features of the other trails
    #rec_index = np.argsort(ed)[0][::-1][1:]
    rec_index = np.argsort(ed)[0][1:]
    print('The sorted euclidean distance measurements are:{0}'.format(np.sort(ed[0]))) #.sort(reverse = True)))
    print('The trail most similiar to the original trail has features: {0}'.format(scaled_features[rec_index][0]))
    #rec_index is an array of the indices of the euclidean distance measurements ordered by greatest to least, leaving out the given trail
    ordered_df = US_trails.loc[rec_index]
    print('The trail most similar to the original trail is: {0}'.format(ordered_df.iloc[0]['name']))
    #ordered_df gives the trails of US_trails ordered by the trails most similar to least similar
    if desired_state is not None:
        ordered_df = ordered_df[ordered_df['state']== desired_state]
    #if a desired state to ride in is given, the dataframe is returned with trails only in that state
    if desired_city_town is not None:
        ordered_df = ordered_df[ordered_df['city/town']== desired_city_town]
    rec_df = ordered_df.head(20)
    #rec_df is a dataframe with the number of desired trails to be recommended
    orig_row = US_trails.loc[[index]].rename(lambda x: 'original')
    #orig_row renames the index of the original trail given as 'original'
    total = pd.concat([orig_row, rec_df])
    #this combines the original row labeled as original row and the n recommended trails that are most similar to the original
    return total

In [437]:
cos_sim_recommendations("The Whole Enchilada", 'Utah', Big_X)

The scaled features of the original trail are:[[ 0.90565559 -7.60488757  1.6195909   3.51553271  2.6075075  -2.6075075 ]]
The sorted cosine similarity measurements are:[ 1.          0.99253285  0.98917966 ..., -0.91430571 -0.9143526
 -0.91440157]
The trail most similiar to the original trail has features: [ 1.37713107 -7.85023322  1.6195909   2.50782783  2.6075075  -2.6075075 ]
The trail most similar to the original trail is: Cannell Trail - IMBA EPIC


,name,location,difficulty,length,ascent,descent,stars,latitude,longitude,summary,...,type_Trail,city/town,state,length_range,ascent_scaled,descent_scaled,difficulty_encoded_scaled,length_scaled,type_Featured Ride_scaled,type_Trail_scaled
original,The Whole Enchilada,"Moab, Utah",Black,34.1,1283,-7794,4.8,38.4855,-109.2320,"One of the world's great rides, with 8000 ft o...",...,0,Moab,Utah,30+,0.905656,-7.604888,1.619591,3.515533,2.607508,-2.607508
4551,Cannell Trail - IMBA EPIC,"Kernville, California",Black,25.5,1718,-8030,4.6,35.9905,-118.3657,This epic Southern California jewel is a true ...,...,0,Kernville,California,25-30,1.377131,-7.850233,1.619591,2.507828,2.607508,-2.607508
1332,CDO,"Catalina, Arizona",Black,21.1,1105,-6939,3.6,32.4407,-110.7861,An amazing trail traversing from the top of Mt...,...,0,Catalina,Arizona,20-25,0.712730,-6.716029,1.619591,1.992258,2.607508,-2.607508
2933,Bike For Bender Hell Ride,"Big Bear City, California",Black,35.9,2349,-7055,4.8,34.1707,-116.8299,A point-to-point adventure from South Fork Cam...,...,0,Big Bear City,California,30+,2.061041,-6.836623,1.619591,3.726448,2.607508,-2.607508
8132,Never Eat Soggy Waffels,"Junction, Utah",Black,22.3,1778,-6709,4.0,38.3790,-112.4002,Shuttled ride from mud lake down to Beaver lin...,...,0,Junction,Utah,20-25,1.442162,-6.476921,1.619591,2.132868,2.607508,-2.607508
10089,Big Boulder from Packer Saddle,"Downieville, California",Black,21.8,1644,-5813,4.3,39.6145,-120.6666,You still have about 2000ft of climbing on you...,...,0,Downieville,California,20-25,1.296926,-5.545440,1.619591,2.074280,2.607508,-2.607508
1038,Full Lemmon Drop,"Catalina, Arizona",Double Black,29.4,2999,-9281,3.1,32.4404,-110.7856,"High alpine to the desert floor - a 9,000' dif...",...,0,Catalina,Arizona,25-30,2.765545,-9.150773,2.404446,2.964810,2.607508,-2.607508
1088,Lemmon Drop,"Tanque Verde, Arizona",Black,18.8,1928,-6457,4.1,32.3980,-110.6893,"A long, mostly downhill tour of Mt. Lemmon.",...,0,Tanque Verde,Arizona,15-20,1.604740,-6.214942,1.619591,1.722755,2.607508,-2.607508
8361,Monarch Crest - IMBA EPIC,"Whitepine, Colorado",Blue/Black,36.2,2035,-5890,4.7,38.4965,-106.3254,One of Colorado's top 5 epic rides - classic h...,...,0,Whitepine,Colorado,30+,1.720712,-5.625489,0.834736,3.761600,2.607508,-2.607508
15995,Paulina Peak to Horse Butte,"La Pine, Oregon",Blue/Black,38.7,1878,-5842,4.0,43.6895,-121.2545,An amazing tour of the Newberry Crater with a ...,...,0,La Pine,Oregon,30+,1.550547,-5.575588,0.834736,4.054538,2.607508,-2.607508


In [438]:
def cosim(v1, v2):
    dot_product = sum(n1 * n2 for n1, n2 in zip(v1, v2) )
    magnitude1 = math.sqrt(sum(n ** 2 for n in v1))
    magnitude2 = math.sqrt(sum(n ** 2 for n in v2))
    return dot_product / (magnitude1 * magnitude2)

In [439]:
v1 = [0.90565559, -7.60488757,  1.6195909,   3.51553271,  1.26812186,  2.6075075, -2.6075075]
v2 = [1.37713107, -7.85023322, 1.6195909, 2.50782783, 1.06278865, 2.6075075, -2.6075075]
cosim(v1, v2)

0.9924311777433537

In [440]:
euclidean_dist_recommendations("The Whole Enchilada", 'Utah', Big_X)

The scaled features of the original trail are:[[ 0.90565559 -7.60488757  1.6195909   3.51553271  2.6075075  -2.6075075 ]]
The sorted euclidean distance measurements are:[  0.           1.13927728   1.40343582 ...,  37.14330007  37.53581505
  47.82939655]
The trail most similiar to the original trail has features: [ 1.37713107 -7.85023322  1.6195909   2.50782783  2.6075075  -2.6075075 ]
The trail most similar to the original trail is: Cannell Trail - IMBA EPIC


,name,location,difficulty,length,ascent,descent,stars,latitude,longitude,summary,...,type_Trail,city/town,state,length_range,ascent_scaled,descent_scaled,difficulty_encoded_scaled,length_scaled,type_Featured Ride_scaled,type_Trail_scaled
original,The Whole Enchilada,"Moab, Utah",Black,34.1,1283,-7794,4.8,38.4855,-109.2320,"One of the world's great rides, with 8000 ft o...",...,0,Moab,Utah,30+,0.905656,-7.604888,1.619591,3.515533,2.607508,-2.607508
4551,Cannell Trail - IMBA EPIC,"Kernville, California",Black,25.5,1718,-8030,4.6,35.9905,-118.3657,This epic Southern California jewel is a true ...,...,0,Kernville,California,25-30,1.377131,-7.850233,1.619591,2.507828,2.607508,-2.607508
2933,Bike For Bender Hell Ride,"Big Bear City, California",Black,35.9,2349,-7055,4.8,34.1707,-116.8299,A point-to-point adventure from South Fork Cam...,...,0,Big Bear City,California,30+,2.061041,-6.836623,1.619591,3.726448,2.607508,-2.607508
1332,CDO,"Catalina, Arizona",Black,21.1,1105,-6939,3.6,32.4407,-110.7861,An amazing trail traversing from the top of Mt...,...,0,Catalina,Arizona,20-25,0.712730,-6.716029,1.619591,1.992258,2.607508,-2.607508
8132,Never Eat Soggy Waffels,"Junction, Utah",Black,22.3,1778,-6709,4.0,38.3790,-112.4002,Shuttled ride from mud lake down to Beaver lin...,...,0,Junction,Utah,20-25,1.442162,-6.476921,1.619591,2.132868,2.607508,-2.607508
8361,Monarch Crest - IMBA EPIC,"Whitepine, Colorado",Blue/Black,36.2,2035,-5890,4.7,38.4965,-106.3254,One of Colorado's top 5 epic rides - classic h...,...,0,Whitepine,Colorado,30+,1.720712,-5.625489,0.834736,3.761600,2.607508,-2.607508
15995,Paulina Peak to Horse Butte,"La Pine, Oregon",Blue/Black,38.7,1878,-5842,4.0,43.6895,-121.2545,An amazing tour of the Newberry Crater with a ...,...,0,La Pine,Oregon,30+,1.550547,-5.575588,0.834736,4.054538,2.607508,-2.607508
1088,Lemmon Drop,"Tanque Verde, Arizona",Black,18.8,1928,-6457,4.1,32.3980,-110.6893,"A long, mostly downhill tour of Mt. Lemmon.",...,0,Tanque Verde,Arizona,15-20,1.604740,-6.214942,1.619591,1.722755,2.607508,-2.607508
12,Haleakala Shuttle,"Kēōkea, Hawaii",Black,17.8,186,-7043,5.0,20.7086,-156.2540,An epic descent from the rim of the Haleakala ...,...,0,Kēōkea,Hawaii,15-20,-0.283330,-6.824148,1.619591,1.605580,2.607508,-2.607508
10089,Big Boulder from Packer Saddle,"Downieville, California",Black,21.8,1644,-5813,4.3,39.6145,-120.6666,You still have about 2000ft of climbing on you...,...,0,Downieville,California,20-25,1.296926,-5.545440,1.619591,2.074280,2.607508,-2.607508


In [441]:
def euc_dist(x,y):   
    dist = [(a - b)**2 for a, b in zip(x, y)]
    dist = math.sqrt(sum(dist))
    return dist

In [442]:
x = [0.90565559, -7.60488757,  1.6195909,   3.51553271,  1.26812186,  2.6075075, -2.6075075]
y = [1.37713107, -7.85023322,  1.6195909,   2.50782783,  1.06278865,  2.6075075, -2.6075075 ]
euc_dist(x,y)

1.1576331320931823

In [443]:
def list_to_npArray(vector1, vector2):
    '''convert the list to numpy array'''
    if type(vector1) == list:
        vector1 = np.array(vector1)
    if type(vector2) == list:
        vector2 = np.array(vector2)
    return vector1, vector2

def np_euc_dist(vector1,vector2): 
    vector1, vector2 = list_to_npArray(vector1, vector2)
    distance = np.linalg.norm(vector1-vector2, 2, 0)
    return distance
np_euc_dist(x,y)

1.1576331320931823

In [444]:
cos_sim_recommendations("Porcupine Rim", 'Utah', Big_X)

The scaled features of the original trail are:[[ 0.81027664 -2.58153942  1.6195909   1.242338    2.6075075  -2.6075075 ]]
The sorted cosine similarity measurements are:[ 1.          0.99395382  0.99278688 ..., -0.91998949 -0.92060637
 -0.92072064]
The trail most similiar to the original trail has features: [ 0.3669813  -2.91213228  1.6195909   1.12516301  2.6075075  -2.6075075 ]
The trail most similar to the original trail is: Monarch to Fooses Creek


,name,location,difficulty,length,ascent,descent,stars,latitude,longitude,summary,...,type_Trail,city/town,state,length_range,ascent_scaled,descent_scaled,difficulty_encoded_scaled,length_scaled,type_Featured Ride_scaled,type_Trail_scaled
original,Porcupine Rim,"Moab, Utah",Black,14.7,1195,-2962,4.7,38.5819,-109.4164,The world famous Porcupine Rim ride; a bone ja...,...,0,Moab,Utah,10-15,0.810277,-2.581539,1.619591,1.242338,2.607508,-2.607508
8363,Monarch to Fooses Creek,"Salida, Colorado",Black,13.7,786,-3280,4.4,38.4960,-106.3247,Beautiful views on the start to the Monarch Cr...,...,0,Salida,Colorado,10-15,0.366981,-2.912132,1.619591,1.125163,2.607508,-2.607508
13752,White Pine to Bunchgrass,"Richmond, Utah",Black,10.7,886,-2663,3.6,41.8952,-111.6425,A short climb that leads to Cache Valley's bes...,...,0,Richmond,Utah,10-15,0.475366,-2.270699,1.619591,0.773638,2.607508,-2.607508
18699,Idaho Trail Segment 17,"Grangeville, Idaho",Black,15.4,1795,-3128,3.0,45.8826,-115.2326,Idaho Trail Singletrack Mountain Segment,...,0,Grangeville,Idaho,15-20,1.460588,-2.754113,1.619591,1.324360,2.607508,-2.607508
7703,Snow Mesa to Miner's Creek,"Lake City, Colorado",Black,16.7,1686,-3550,4.6,37.9397,-107.1587,A sublime ride experience starting on an alpin...,...,0,Lake City,Colorado,15-20,1.342448,-3.192824,1.619591,1.476688,2.607508,-2.607508
1265,BLT to Noble Canyon,"Pine Valley, California",Black,20.2,1240,-3506,4.7,32.8570,-116.4241,"A one-of-a-kind ride through the ridges, meado...",...,0,Pine Valley,California,20-25,0.859050,-3.147082,1.619591,1.886800,2.607508,-2.607508
8365,Monarch Crest to Green's,"Salida, Colorado",Black,14.0,673,-3417,5.0,38.4964,-106.3253,A great combination of the scenic Continental ...,...,0,Salida,Colorado,10-15,0.244506,-3.054558,1.619591,1.160316,2.607508,-2.607508
8946,TRT-Flume-Chimney Beach Exit,"Incline Village, Nevada",Black,19.7,1485,-3735,5.0,39.3074,-119.9081,Classic TRT-Flume point to point ride. Leave ...,...,0,Incline Village,Nevada,15-20,1.124594,-3.385150,1.619591,1.828213,2.607508,-2.607508
18598,Falls Creek Shuttle,"Stevenson, Washington",Blue/Black,15.4,1032,-2744,4.4,45.9900,-121.9131,Being written,...,0,Stevenson,Washington,15-20,0.633609,-2.354907,0.834736,1.324360,2.607508,-2.607508
18713,Bradley Gulch to Sheep Creek,"Salmon, Idaho",Black,14.5,1161,-4016,4.0,45.5495,-113.8210,Classic backcountry riding along Idaho's Conti...,...,0,Salmon,Idaho,10-15,0.773426,-3.677278,1.619591,1.218903,2.607508,-2.607508


In [445]:
euclidean_dist_recommendations("Porcupine Rim", 'Utah', Big_X)

The scaled features of the original trail are:[[ 0.81027664 -2.58153942  1.6195909   1.242338    2.6075075  -2.6075075 ]]
The sorted euclidean distance measurements are:[  0.           0.56527195   0.65457333 ...,  41.07872797  41.62610257
  51.81477979]
The trail most similiar to the original trail has features: [ 0.3669813  -2.91213228  1.6195909   1.12516301  2.6075075  -2.6075075 ]
The trail most similar to the original trail is: Monarch to Fooses Creek


,name,location,difficulty,length,ascent,descent,stars,latitude,longitude,summary,...,type_Trail,city/town,state,length_range,ascent_scaled,descent_scaled,difficulty_encoded_scaled,length_scaled,type_Featured Ride_scaled,type_Trail_scaled
original,Porcupine Rim,"Moab, Utah",Black,14.7,1195,-2962,4.7,38.5819,-109.4164,The world famous Porcupine Rim ride; a bone ja...,...,0,Moab,Utah,10-15,0.810277,-2.581539,1.619591,1.242338,2.607508,-2.607508
8363,Monarch to Fooses Creek,"Salida, Colorado",Black,13.7,786,-3280,4.4,38.4960,-106.3247,Beautiful views on the start to the Monarch Cr...,...,0,Salida,Colorado,10-15,0.366981,-2.912132,1.619591,1.125163,2.607508,-2.607508
13752,White Pine to Bunchgrass,"Richmond, Utah",Black,10.7,886,-2663,3.6,41.8952,-111.6425,A short climb that leads to Cache Valley's bes...,...,0,Richmond,Utah,10-15,0.475366,-2.270699,1.619591,0.773638,2.607508,-2.607508
18699,Idaho Trail Segment 17,"Grangeville, Idaho",Black,15.4,1795,-3128,3.0,45.8826,-115.2326,Idaho Trail Singletrack Mountain Segment,...,0,Grangeville,Idaho,15-20,1.460588,-2.754113,1.619591,1.324360,2.607508,-2.607508
8365,Monarch Crest to Green's,"Salida, Colorado",Black,14.0,673,-3417,5.0,38.4964,-106.3253,A great combination of the scenic Continental ...,...,0,Salida,Colorado,10-15,0.244506,-3.054558,1.619591,1.160316,2.607508,-2.607508
18598,Falls Creek Shuttle,"Stevenson, Washington",Blue/Black,15.4,1032,-2744,4.4,45.9900,-121.9131,Being written,...,0,Stevenson,Washington,15-20,0.633609,-2.354907,0.834736,1.324360,2.607508,-2.607508
7703,Snow Mesa to Miner's Creek,"Lake City, Colorado",Black,16.7,1686,-3550,4.6,37.9397,-107.1587,A sublime ride experience starting on an alpin...,...,0,Lake City,Colorado,15-20,1.342448,-3.192824,1.619591,1.476688,2.607508,-2.607508
1265,BLT to Noble Canyon,"Pine Valley, California",Black,20.2,1240,-3506,4.7,32.8570,-116.4241,"A one-of-a-kind ride through the ridges, meado...",...,0,Pine Valley,California,20-25,0.859050,-3.147082,1.619591,1.886800,2.607508,-2.607508
8947,Armstrong Pass Ride,"South Lake Tahoe, California",Blue/Black,16.6,1439,-2669,4.7,38.7913,-119.9325,An awesome ride with 12.0 miles of downhill sw...,...,0,South Lake Tahoe,California,15-20,1.074736,-2.276937,0.834736,1.464970,2.607508,-2.607508
7730,St. Charles East (South),"Colorado City, Colorado",Black,8.6,675,-3204,3.3,38.0135,-105.1385,"A rowdy, adventurous shuttle drop in the Wet M...",...,0,Colorado City,Colorado,5-10,0.246674,-2.833123,1.619591,0.527571,2.607508,-2.607508


In [446]:
cos_sim_recommendations("Mount Falcon and Lair O' the Bear Loop", 'Colorado', Big_X)
#COMPARE Gunsight Ridge TO Boulder Lake Loop IN SAYING WHY COS SIM IS BETTER

The scaled features of the original trail are:[[ 2.39270009 -2.26758016  0.83473587  1.78134293  2.6075075  -2.6075075 ]]
The sorted cosine similarity measurements are:[ 1.          0.99981842  0.99970404 ..., -0.97536432 -0.97541078
 -0.97546761]
The trail most similiar to the original trail has features: [ 2.47832438 -2.32891657  0.83473587  1.87508292  2.6075075  -2.6075075 ]
The trail most similar to the original trail is: Doctor Park


,name,location,difficulty,length,ascent,descent,stars,latitude,longitude,summary,...,type_Trail,city/town,state,length_range,ascent_scaled,descent_scaled,difficulty_encoded_scaled,length_scaled,type_Featured Ride_scaled,type_Trail_scaled
original,Mount Falcon and Lair O' the Bear Loop,"Morrison, Colorado",Blue/Black,19.3,2655,-2660,3.6,39.6468,-105.1968,A physically challenging but technically moder...,...,0,Morrison,Colorado,15-20,2.392700,-2.267580,0.834736,1.781343,2.607508,-2.607508
9344,Doctor Park,"Gunnison, Colorado",Blue/Black,20.1,2734,-2719,4.7,38.7301,-106.7580,A local favorite with a tough grind of a climb...,...,0,Gunnison,Colorado,20-25,2.478324,-2.328917,0.834736,1.875083,2.607508,-2.607508
18165,Gunsight Ridge,"Government Camp, Oregon",Blue/Black,18.7,2550,-2548,4.8,45.3101,-121.6418,A rigorous ridge trail that has spectacular vi...,...,0,Government Camp,Oregon,15-20,2.278896,-2.151145,0.834736,1.711038,2.607508,-2.607508
18168,Boulder Lake Loop,"Government Camp, Oregon",Blue/Black,17.9,2676,-2674,3.7,45.3102,-121.6419,An adventurous loop ride to Boulder Lake via G...,...,0,Government Camp,Oregon,15-20,2.415461,-2.282135,0.834736,1.617298,2.607508,-2.607508
16120,Double D,"Boise, Idaho",Blue/Black,17.7,2596,-2594,4.0,43.6391,-116.1946,The best of the central foothills - Bob's and ...,...,0,Boise,Idaho,15-20,2.328753,-2.198967,0.834736,1.593863,2.607508,-2.607508
18938,Buttercup Loop,"Hamilton, Montana",Blue/Black,21.1,2776,-2781,4.0,46.1538,-114.1289,"Some pavement, some gravel, and a fantastic do...",...,0,Hamilton,Montana,20-25,2.523846,-2.393372,0.834736,1.992258,2.607508,-2.607508
19930,Bernard Peak - Scout Trail Option,"Spirit Lake, Idaho",Blue/Black,18.3,2782,-2786,4.3,47.9469,-116.6128,A fun outing with a flowy forested downhill fr...,...,0,Spirit Lake,Idaho,15-20,2.530349,-2.398570,0.834736,1.664168,2.607508,-2.607508
8663,Braley's Pond & Confederate Breastworks Loop,"Weyers Cave, Virginia",Blue/Black,18.0,2753,-2758,4.0,38.2866,-79.3009,The first 1/3 of the ride is fireroad and pave...,...,0,Weyers Cave,Virginia,15-20,2.498918,-2.369461,0.834736,1.629015,2.607508,-2.607508
2756,Chino Hills Loop,"Yorba Linda, California",Blue/Black,20.0,2855,-2854,3.9,33.9090,-117.7801,A loop ride to get the most out of Chino Hills.,...,0,Yorba Linda,California,15-20,2.609470,-2.469263,0.834736,1.863365,2.607508,-2.607508
7053,Potts Mountain Jeep Trail Loop,"New Castle, Virginia",Blue/Black,18.1,2475,-2472,3.0,37.6051,-80.0758,A scenic route that circumnavigates Barbours C...,...,0,New Castle,Virginia,15-20,2.197607,-2.072135,0.834736,1.640733,2.607508,-2.607508


In [447]:
euclidean_dist_recommendations("Mount Falcon and Lair O' the Bear Loop", 'Colorado', Big_X)

The scaled features of the original trail are:[[ 2.39270009 -2.26758016  0.83473587  1.78134293  2.6075075  -2.6075075 ]]
The sorted euclidean distance measurements are:[  0.           0.1409995    0.16625476 ...,  40.01463102  40.59435193
  50.74949838]
The trail most similiar to the original trail has features: [ 2.47832438 -2.32891657  0.83473587  1.87508292  2.6075075  -2.6075075 ]
The trail most similar to the original trail is: Doctor Park


,name,location,difficulty,length,ascent,descent,stars,latitude,longitude,summary,...,type_Trail,city/town,state,length_range,ascent_scaled,descent_scaled,difficulty_encoded_scaled,length_scaled,type_Featured Ride_scaled,type_Trail_scaled
original,Mount Falcon and Lair O' the Bear Loop,"Morrison, Colorado",Blue/Black,19.3,2655,-2660,3.6,39.6468,-105.1968,A physically challenging but technically moder...,...,0,Morrison,Colorado,15-20,2.392700,-2.267580,0.834736,1.781343,2.607508,-2.607508
9344,Doctor Park,"Gunnison, Colorado",Blue/Black,20.1,2734,-2719,4.7,38.7301,-106.7580,A local favorite with a tough grind of a climb...,...,0,Gunnison,Colorado,20-25,2.478324,-2.328917,0.834736,1.875083,2.607508,-2.607508
18168,Boulder Lake Loop,"Government Camp, Oregon",Blue/Black,17.9,2676,-2674,3.7,45.3102,-121.6419,An adventurous loop ride to Boulder Lake via G...,...,0,Government Camp,Oregon,15-20,2.415461,-2.282135,0.834736,1.617298,2.607508,-2.607508
18165,Gunsight Ridge,"Government Camp, Oregon",Blue/Black,18.7,2550,-2548,4.8,45.3101,-121.6418,A rigorous ridge trail that has spectacular vi...,...,0,Government Camp,Oregon,15-20,2.278896,-2.151145,0.834736,1.711038,2.607508,-2.607508
16120,Double D,"Boise, Idaho",Blue/Black,17.7,2596,-2594,4.0,43.6391,-116.1946,The best of the central foothills - Bob's and ...,...,0,Boise,Idaho,15-20,2.328753,-2.198967,0.834736,1.593863,2.607508,-2.607508
8663,Braley's Pond & Confederate Breastworks Loop,"Weyers Cave, Virginia",Blue/Black,18.0,2753,-2758,4.0,38.2866,-79.3009,The first 1/3 of the ride is fireroad and pave...,...,0,Weyers Cave,Virginia,15-20,2.498918,-2.369461,0.834736,1.629015,2.607508,-2.607508
19930,Bernard Peak - Scout Trail Option,"Spirit Lake, Idaho",Blue/Black,18.3,2782,-2786,4.3,47.9469,-116.6128,A fun outing with a flowy forested downhill fr...,...,0,Spirit Lake,Idaho,15-20,2.530349,-2.398570,0.834736,1.664168,2.607508,-2.607508
18938,Buttercup Loop,"Hamilton, Montana",Blue/Black,21.1,2776,-2781,4.0,46.1538,-114.1289,"Some pavement, some gravel, and a fantastic do...",...,0,Hamilton,Montana,20-25,2.523846,-2.393372,0.834736,1.992258,2.607508,-2.607508
2666,New Millennium Trail + Las Virgenes Downhill,"Agoura, California",Blue/Black,17.2,2558,-2568,4.1,34.1341,-118.7025,"An amazing loop with awesome views, technical ...",...,0,Agoura,California,15-20,2.287566,-2.171937,0.834736,1.535275,2.607508,-2.607508
9282,409s and Point Lookout Loop,"Crested Butte, Colorado",Blue/Black,16.9,2708,-2709,4.2,38.8644,-106.9130,"Great views, a fun, moderately technical desce...",...,0,Crested Butte,Colorado,15-20,2.450144,-2.318521,0.834736,1.500123,2.607508,-2.607508


In [448]:
cos_sim_recommendations("Enchanted Forest", 'Colorado', Big_X)

The scaled features of the original trail are:[[-0.43181738 -0.01684573  0.83473587 -0.30437181 -0.38350801  0.38350801]]
The sorted cosine similarity measurements are:[ 1.          0.99986543  0.99984179 ..., -0.81111992 -0.81476252
 -0.82078886]
The trail most similiar to the original trail has features: [-0.42423042 -0.00333093  0.83473587 -0.29265431 -0.38350801  0.38350801]
The trail most similar to the original trail is: Deadline Trail


,name,location,difficulty,length,ascent,descent,stars,latitude,longitude,summary,...,type_Trail,city/town,state,length_range,ascent_scaled,descent_scaled,difficulty_encoded_scaled,length_scaled,type_Featured Ride_scaled,type_Trail_scaled
original,Enchanted Forest,"Golden, Colorado",Blue/Black,1.5,49,-495,4.7,39.7173,-105.2483,A super fun ride through the trees as an alter...,...,1,Golden,Colorado,0-5,-0.431817,-0.016846,0.834736,-0.304372,-0.383508,0.383508
10339,Deadline Trail,"Snowmass Village, Colorado",Blue/Black,1.6,56,-482,5.0,39.2199,-106.9101,Bike optimized downhill recommended flow style...,...,1,Snowmass Village,Colorado,0-5,-0.424230,-0.003331,0.834736,-0.292654,-0.383508,0.383508
11779,Rattlesnake Ridge Trail,"Shippensburg, Pennsylvania",Blue/Black,1.4,62,-503,3.0,40.0258,-77.3554,Needs Summary,...,1,Shippensburg,Pennsylvania,0-5,-0.417727,-0.025163,0.834736,-0.316089,-0.383508,0.383508
957,Vertigo Ridge,"El Paso, Texas",Blue/Black,1.3,36,-498,4.8,31.8035,-106.4925,"Fast, technical switchbacks and singletrack fr...",...,1,El Paso,Texas,0-5,-0.445907,-0.019965,0.834736,-0.327807,-0.383508,0.383508
19791,Summit Trail,"Mirrormont, Washington",Blue/Black,1.5,18,-510,4.3,47.4882,-121.9473,Fast pump-track feel through the woods.,...,1,Mirrormont,Washington,0-5,-0.465417,-0.032440,0.834736,-0.304372,-0.383508,0.383508
17981,Douty Mill Trail,"Mifflinburg, Pennsylvania",Blue/Black,1.2,60,-507,3.0,40.9692,-77.2421,"A rough, rocky descent from Fallen Timber Trai...",...,1,Mifflinburg,Pennsylvania,0-5,-0.419895,-0.029321,0.834736,-0.339524,-0.383508,0.383508
4352,Rocky Ridge Trail,"Etowah, North Carolina",Blue/Black,1.5,27,-464,4.5,35.2119,-82.5731,A twisty winding descent from Sky Valley Road ...,...,1,Etowah,North Carolina,0-5,-0.455662,0.015382,0.834736,-0.304372,-0.383508,0.383508
4865,Ingles Field Gap,"Bent Creek, North Carolina",Blue/Black,1.7,39,-523,4.4,35.4976,-82.6481,"A fast, fun downhill with jumps, a couple berm...",...,1,Bent Creek,North Carolina,0-5,-0.442656,-0.045955,0.834736,-0.280937,-0.383508,0.383508
8741,Middle Ridge Trail (Stinky Pete to Davis Creek),"Rand, West Virginia",Blue/Black,1.2,60,-477,4.0,38.2742,-81.6514,The newest section of the MRT system. A single...,...,1,Rand,West Virginia,0-5,-0.419895,0.001867,0.834736,-0.339524,-0.383508,0.383508
10784,Charit Creek Horse Trail,"Oneida, Tennessee",Blue/Black,1.1,34,-492,2.0,36.5287,-84.7291,This is a STEEP gravel road granting horse/bik...,...,1,Oneida,Tennessee,0-5,-0.448075,-0.013727,0.834736,-0.351242,-0.383508,0.383508


In [449]:
euclidean_dist_recommendations("Enchanted Forest", 'Colorado', Big_X)
#it seems that euclidean is more likely to give back trails with shorter distances but greater absolute value in sum of ascent and descent

The scaled features of the original trail are:[[-0.43181738 -0.01684573  0.83473587 -0.30437181 -0.38350801  0.38350801]]
The sorted euclidean distance measurements are:[  0.00000000e+00   1.94296590e-02   2.01245890e-02 ...,   4.38847610e+01
   4.44908958e+01   5.50360526e+01]
The trail most similiar to the original trail has features: [-0.42423042 -0.00333093  0.83473587 -0.29265431 -0.38350801  0.38350801]
The trail most similar to the original trail is: Deadline Trail


,name,location,difficulty,length,ascent,descent,stars,latitude,longitude,summary,...,type_Trail,city/town,state,length_range,ascent_scaled,descent_scaled,difficulty_encoded_scaled,length_scaled,type_Featured Ride_scaled,type_Trail_scaled
original,Enchanted Forest,"Golden, Colorado",Blue/Black,1.5,49,-495,4.7,39.7173,-105.2483,A super fun ride through the trees as an alter...,...,1,Golden,Colorado,0-5,-0.431817,-0.016846,0.834736,-0.304372,-0.383508,0.383508
10339,Deadline Trail,"Snowmass Village, Colorado",Blue/Black,1.6,56,-482,5.0,39.2199,-106.9101,Bike optimized downhill recommended flow style...,...,1,Snowmass Village,Colorado,0-5,-0.424230,-0.003331,0.834736,-0.292654,-0.383508,0.383508
11779,Rattlesnake Ridge Trail,"Shippensburg, Pennsylvania",Blue/Black,1.4,62,-503,3.0,40.0258,-77.3554,Needs Summary,...,1,Shippensburg,Pennsylvania,0-5,-0.417727,-0.025163,0.834736,-0.316089,-0.383508,0.383508
957,Vertigo Ridge,"El Paso, Texas",Blue/Black,1.3,36,-498,4.8,31.8035,-106.4925,"Fast, technical switchbacks and singletrack fr...",...,1,El Paso,Texas,0-5,-0.445907,-0.019965,0.834736,-0.327807,-0.383508,0.383508
19791,Summit Trail,"Mirrormont, Washington",Blue/Black,1.5,18,-510,4.3,47.4882,-121.9473,Fast pump-track feel through the woods.,...,1,Mirrormont,Washington,0-5,-0.465417,-0.032440,0.834736,-0.304372,-0.383508,0.383508
4865,Ingles Field Gap,"Bent Creek, North Carolina",Blue/Black,1.7,39,-523,4.4,35.4976,-82.6481,"A fast, fun downhill with jumps, a couple berm...",...,1,Bent Creek,North Carolina,0-5,-0.442656,-0.045955,0.834736,-0.280937,-0.383508,0.383508
17981,Douty Mill Trail,"Mifflinburg, Pennsylvania",Blue/Black,1.2,60,-507,3.0,40.9692,-77.2421,"A rough, rocky descent from Fallen Timber Trai...",...,1,Mifflinburg,Pennsylvania,0-5,-0.419895,-0.029321,0.834736,-0.339524,-0.383508,0.383508
4352,Rocky Ridge Trail,"Etowah, North Carolina",Blue/Black,1.5,27,-464,4.5,35.2119,-82.5731,A twisty winding descent from Sky Valley Road ...,...,1,Etowah,North Carolina,0-5,-0.455662,0.015382,0.834736,-0.304372,-0.383508,0.383508
8741,Middle Ridge Trail (Stinky Pete to Davis Creek),"Rand, West Virginia",Blue/Black,1.2,60,-477,4.0,38.2742,-81.6514,The newest section of the MRT system. A single...,...,1,Rand,West Virginia,0-5,-0.419895,0.001867,0.834736,-0.339524,-0.383508,0.383508
10784,Charit Creek Horse Trail,"Oneida, Tennessee",Blue/Black,1.1,34,-492,2.0,36.5287,-84.7291,This is a STEEP gravel road granting horse/bik...,...,1,Oneida,Tennessee,0-5,-0.448075,-0.013727,0.834736,-0.351242,-0.383508,0.383508


In [453]:
cos_sim_recommendations("Hermosa Creek Trail", 'Colorado', Big_X)

The scaled features of the original trail are:[[ 0.79618657 -1.9276309   0.83473587  1.69932044 -0.38350801  0.38350801]]
The sorted cosine similarity measurements are:[ 1.          0.9826991   0.97583144 ..., -0.69251097 -0.6941739
 -0.69449205]
The trail most similiar to the original trail has features: [ 0.50571432 -1.34025678  0.83473587  0.97283553 -0.38350801  0.38350801]
The trail most similar to the original trail is: Surveyor's Ridge Trail


,name,location,difficulty,length,ascent,descent,stars,latitude,longitude,summary,...,type_Trail,city/town,state,length_range,ascent_scaled,descent_scaled,difficulty_encoded_scaled,length_scaled,type_Featured Ride_scaled,type_Trail_scaled
original,Hermosa Creek Trail,"Rico, Colorado",Blue/Black,18.6,1182,-2333,4.1,37.6299,-107.9158,One of Durango's best - a scenic ride followin...,...,1,Rico,Colorado,15-20,0.796187,-1.927631,0.834736,1.699320,-0.383508,0.383508
18170,Surveyor's Ridge Trail,"Odell, Oregon",Blue/Black,12.4,914,-1768,4.6,45.4088,-121.5496,"One of the top trails in the state of Oregon, ...",...,1,Odell,Oregon,10-15,0.505714,-1.340257,0.834736,0.972836,-0.383508,0.383508
937,AZT - Passage 6 - Las Colinas,"Corona de Tucson, Arizona",Blue/Black,13.3,1295,-2429,2.7,31.8106,-110.7235,This passage extends from Fish Canyon to Santa...,...,1,Corona de Tucson,Arizona,10-15,0.918662,-2.027433,0.834736,1.078293,-0.383508,0.383508
8376,CT - Segment 13,"Buena Vista, Colorado",Blue/Black,16.0,1706,-2672,4.0,38.8129,-106.2799,Needs Summary,...,1,Buena Vista,Colorado,15-20,1.364125,-2.280055,0.834736,1.394665,-0.383508,0.383508
8308,Tabeguache Trail - Roubideau Trail Section,"Montrose, Colorado",Blue/Black,22.1,2190,-3470,4.0,38.3625,-108.2505,This section of the Tabeguache Trail offers lo...,...,1,Montrose,Colorado,20-25,1.888709,-3.109656,0.834736,2.109433,-0.383508,0.383508
18228,Western Rim National Recreation Trail #1774,"Joseph, Oregon",Black,23.0,1452,-2226,5.0,45.3990,-116.7324,A long ridgetop adventure above Hells Canyon N...,...,1,Joseph,Oregon,20-25,1.088827,-1.816394,1.619591,2.214890,-0.383508,0.383508
1918,Skyway Motorway - FS Road 600,"Goodwater, Alabama",Blue/Black,14.3,1366,-1595,4.0,33.1957,-86.0634,Bring extra legs on this stretch of the Skyway...,...,1,Goodwater,Alabama,10-15,0.995615,-1.160406,0.834736,1.195468,-0.383508,0.383508
12033,Coyote Trail,"Heber, Utah",Blue/Black,16.6,1837,-2159,3.4,40.5738,-111.4177,Needs adoption!,...,1,Heber,Utah,15-20,1.506109,-1.746740,0.834736,1.464970,-0.383508,0.383508
5895,Iron Mountain Trail,"Marion, Virginia",Blue/Black,20.8,2223,-3864,4.4,36.7112,-81.4924,Backcountry ridgeline riding in VA.,...,1,Marion,Virginia,20-25,1.924476,-3.519259,0.834736,1.957105,-0.383508,0.383508
11991,Wasatch Crest,"Alta, Utah",Blue/Black,12.5,1035,-2595,4.7,40.6138,-111.5710,Classic high altitude singletrack - up there a...,...,1,Alta,Utah,10-15,0.636860,-2.200006,0.834736,0.984553,-0.383508,0.383508


In [454]:
euclidean_dist_recommendations("Hermosa Creek Trail", 'Colorado', Big_X)

The scaled features of the original trail are:[[ 0.79618657 -1.9276309   0.83473587  1.69932044 -0.38350801  0.38350801]]
The sorted euclidean distance measurements are:[  0.           0.64080856   0.73455533 ...,  40.90421871  41.51490245
  52.07955106]
The trail most similiar to the original trail has features: [ 0.91866181 -2.02743251  0.83473587  1.07829302 -0.38350801  0.38350801]
The trail most similar to the original trail is: AZT - Passage 6 - Las Colinas


,name,location,difficulty,length,ascent,descent,stars,latitude,longitude,summary,...,type_Trail,city/town,state,length_range,ascent_scaled,descent_scaled,difficulty_encoded_scaled,length_scaled,type_Featured Ride_scaled,type_Trail_scaled
original,Hermosa Creek Trail,"Rico, Colorado",Blue/Black,18.6,1182,-2333,4.1,37.6299,-107.9158,One of Durango's best - a scenic ride followin...,...,1,Rico,Colorado,15-20,0.796187,-1.927631,0.834736,1.699320,-0.383508,0.383508
937,AZT - Passage 6 - Las Colinas,"Corona de Tucson, Arizona",Blue/Black,13.3,1295,-2429,2.7,31.8106,-110.7235,This passage extends from Fish Canyon to Santa...,...,1,Corona de Tucson,Arizona,10-15,0.918662,-2.027433,0.834736,1.078293,-0.383508,0.383508
8376,CT - Segment 13,"Buena Vista, Colorado",Blue/Black,16.0,1706,-2672,4.0,38.8129,-106.2799,Needs Summary,...,1,Buena Vista,Colorado,15-20,1.364125,-2.280055,0.834736,1.394665,-0.383508,0.383508
12033,Coyote Trail,"Heber, Utah",Blue/Black,16.6,1837,-2159,3.4,40.5738,-111.4177,Needs adoption!,...,1,Heber,Utah,15-20,1.506109,-1.746740,0.834736,1.464970,-0.383508,0.383508
11991,Wasatch Crest,"Alta, Utah",Blue/Black,12.5,1035,-2595,4.7,40.6138,-111.5710,Classic high altitude singletrack - up there a...,...,1,Alta,Utah,10-15,0.636860,-2.200006,0.834736,0.984553,-0.383508,0.383508
20681,Wickersham Creek Trail,"Ester, Alaska",Blue,19.8,1097,-1977,3.0,65.1781,-148.0761,Head out on this trail through black spruce an...,...,1,Ester,Alaska,15-20,0.704059,-1.557533,0.049881,1.839930,-0.383508,0.383508
1918,Skyway Motorway - FS Road 600,"Goodwater, Alabama",Blue/Black,14.3,1366,-1595,4.0,33.1957,-86.0634,Bring extra legs on this stretch of the Skyway...,...,1,Goodwater,Alabama,10-15,0.995615,-1.160406,0.834736,1.195468,-0.383508,0.383508
16782,Stephens 3/4 Around,"Budd Lake, New Jersey",Blue/Black,15.9,1600,-1600,5.0,40.8897,-74.7641,"Fast, flowy, some big uphills and downhills, t...",...,1,Budd Lake,New Jersey,15-20,1.249237,-1.165604,0.834736,1.382948,-0.383508,0.383508
18170,Surveyor's Ridge Trail,"Odell, Oregon",Blue/Black,12.4,914,-1768,4.6,45.4088,-121.5496,"One of the top trails in the state of Oregon, ...",...,1,Odell,Oregon,10-15,0.505714,-1.340257,0.834736,0.972836,-0.383508,0.383508
16515,Victoria Lake Loop 15,"Colonial Pine Hills, South Dakota",Blue/Black,15.0,1867,-1861,4.3,44.0106,-103.3624,Longest loop in the Victoria Lake area through...,...,1,Colonial Pine Hills,South Dakota,10-15,1.538625,-1.436940,0.834736,1.277490,-0.383508,0.383508


In [469]:
#SHOW HOW TO CALCULATE BOTH AND MENTION THAT IT MIGHT BE A DIFFERENCE OF OPINION BUT YOU THINK THAT GETTING A 
#SIMILAR DISTANCE AND PROPORTION OF CLIMBING TO DESCENDING IS BETTER THAN A SHORTER DISTANCE AND MORE CLIMBING/DESCENDING 
cos_sim_recommendations("Dakota Ridge Trail", 'Colorado', Big_X)

The scaled features of the original trail are:[[-0.28658126  0.08399549  1.6195909  -0.29265431 -0.38350801  0.38350801]]
The sorted cosine similarity measurements are:[ 1.          0.99973816  0.99967478 ..., -0.79575811 -0.79681667
 -0.79783655]
The trail most similiar to the original trail has features: [-0.26815578  0.05696588  1.6195909  -0.31608931 -0.38350801  0.38350801]
The trail most similar to the original trail is: Alternate Flight Pattern


,name,location,difficulty,length,ascent,descent,stars,latitude,longitude,summary,...,type_Trail,city/town,state,length_range,ascent_scaled,descent_scaled,difficulty_encoded_scaled,length_scaled,type_Featured Ride_scaled,type_Trail_scaled
original,Dakota Ridge Trail,"Cañon City, Colorado",Black,1.6,183,-398,4.0,38.4621,-105.2516,Ridge top trail with steep descent.,...,1,Cañon City,Colorado,0-5,-0.286581,0.083995,1.619591,-0.292654,-0.383508,0.383508
7236,Alternate Flight Pattern,"West Lynchburg, Virginia",Black,1.4,200,-424,3.0,37.3582,-79.1614,A fun trail with some nice climbs and descents...,...,1,West Lynchburg,Virginia,0-5,-0.268156,0.056966,1.619591,-0.316089,-0.383508,0.383508
3473,Centennial Trail,"Prescott, Arizona",Black,1.8,200,-431,3.0,34.5581,-112.5045,Bring your technical skills for this one. Tigh...,...,1,Prescott,Arizona,0-5,-0.268156,0.049689,1.619591,-0.269219,-0.383508,0.383508
5264,North Bench Trail,"Jasper, Arkansas",Black,1.4,139,-403,4.0,35.8557,-93.5079,A great downhill trail. Upper terminus accesse...,...,1,Jasper,Arkansas,0-5,-0.334271,0.078797,1.619591,-0.316089,-0.383508,0.383508
16597,Burro Trail,"Coal Creek, Colorado",Black,1.5,230,-394,2.0,39.8555,-105.3623,A steep alternate to completing the eastern mo...,...,1,Coal Creek,Colorado,0-5,-0.235640,0.088154,1.619591,-0.304372,-0.383508,0.383508
6499,South Leaf Trail,"Woodside, California",Black,1.6,178,-347,4.0,37.4018,-122.3334,One of the most technically challenging trails...,...,1,Woodside,California,0-5,-0.292001,0.137015,1.619591,-0.292654,-0.383508,0.383508
9090,Pines Campground to Buck Meadows,"Tuolumne City, California",Black,1.7,147,-364,2.3,37.8190,-120.0951,Steep singletrack trails from the Pines Campgr...,...,1,Tuolumne City,California,0-5,-0.325600,0.119342,1.619591,-0.280937,-0.383508,0.383508
7013,Hell Creek Road,"Beattyville, Kentucky",Black,1.7,208,-352,5.0,37.6373,-83.6572,A hellacious crossing!,...,1,Beattyville,Kentucky,0-5,-0.259485,0.131817,1.619591,-0.280937,-0.383508,0.383508
4593,Central Spine,"Jemez Pueblo, New Mexico",Black,1.6,226,-350,3.4,35.5056,-106.8389,A thrilling and technical trail bisecting the ...,...,1,Jemez Pueblo,New Mexico,0-5,-0.239976,0.133896,1.619591,-0.292654,-0.383508,0.383508
1160,Upper Sunset Trail,"Westway, Texas",Black,1.3,164,-463,2.0,31.9310,-106.5144,A singletrack trail in the Tom Mays unit of th...,...,1,Westway,Texas,0-5,-0.307174,0.016421,1.619591,-0.327807,-0.383508,0.383508


In [470]:
euclidean_dist_recommendations("Dakota Ridge Trail", 'Colorado', Big_X)

The scaled features of the original trail are:[[-0.28658126  0.08399549  1.6195909  -0.29265431 -0.38350801  0.38350801]]
The sorted euclidean distance measurements are:[  0.00000000e+00   4.02404883e-02   4.54494699e-02 ...,   4.38738250e+01
   4.44839358e+01   5.49956750e+01]
The trail most similiar to the original trail has features: [-0.26815578  0.05696588  1.6195909  -0.31608931 -0.38350801  0.38350801]
The trail most similar to the original trail is: Alternate Flight Pattern


,name,location,difficulty,length,ascent,descent,stars,latitude,longitude,summary,...,type_Trail,city/town,state,length_range,ascent_scaled,descent_scaled,difficulty_encoded_scaled,length_scaled,type_Featured Ride_scaled,type_Trail_scaled
original,Dakota Ridge Trail,"Cañon City, Colorado",Black,1.6,183,-398,4.0,38.4621,-105.2516,Ridge top trail with steep descent.,...,1,Cañon City,Colorado,0-5,-0.286581,0.083995,1.619591,-0.292654,-0.383508,0.383508
7236,Alternate Flight Pattern,"West Lynchburg, Virginia",Black,1.4,200,-424,3.0,37.3582,-79.1614,A fun trail with some nice climbs and descents...,...,1,West Lynchburg,Virginia,0-5,-0.268156,0.056966,1.619591,-0.316089,-0.383508,0.383508
3473,Centennial Trail,"Prescott, Arizona",Black,1.8,200,-431,3.0,34.5581,-112.5045,Bring your technical skills for this one. Tigh...,...,1,Prescott,Arizona,0-5,-0.268156,0.049689,1.619591,-0.269219,-0.383508,0.383508
16597,Burro Trail,"Coal Creek, Colorado",Black,1.5,230,-394,2.0,39.8555,-105.3623,A steep alternate to completing the eastern mo...,...,1,Coal Creek,Colorado,0-5,-0.235640,0.088154,1.619591,-0.304372,-0.383508,0.383508
6499,South Leaf Trail,"Woodside, California",Black,1.6,178,-347,4.0,37.4018,-122.3334,One of the most technically challenging trails...,...,1,Woodside,California,0-5,-0.292001,0.137015,1.619591,-0.292654,-0.383508,0.383508
5264,North Bench Trail,"Jasper, Arkansas",Black,1.4,139,-403,4.0,35.8557,-93.5079,A great downhill trail. Upper terminus accesse...,...,1,Jasper,Arkansas,0-5,-0.334271,0.078797,1.619591,-0.316089,-0.383508,0.383508
9090,Pines Campground to Buck Meadows,"Tuolumne City, California",Black,1.7,147,-364,2.3,37.8190,-120.0951,Steep singletrack trails from the Pines Campgr...,...,1,Tuolumne City,California,0-5,-0.325600,0.119342,1.619591,-0.280937,-0.383508,0.383508
7013,Hell Creek Road,"Beattyville, Kentucky",Black,1.7,208,-352,5.0,37.6373,-83.6572,A hellacious crossing!,...,1,Beattyville,Kentucky,0-5,-0.259485,0.131817,1.619591,-0.280937,-0.383508,0.383508
4593,Central Spine,"Jemez Pueblo, New Mexico",Black,1.6,226,-350,3.4,35.5056,-106.8389,A thrilling and technical trail bisecting the ...,...,1,Jemez Pueblo,New Mexico,0-5,-0.239976,0.133896,1.619591,-0.292654,-0.383508,0.383508
1160,Upper Sunset Trail,"Westway, Texas",Black,1.3,164,-463,2.0,31.9310,-106.5144,A singletrack trail in the Tom Mays unit of th...,...,1,Westway,Texas,0-5,-0.307174,0.016421,1.619591,-0.327807,-0.383508,0.383508


In [471]:
cos_sim_recommendations("Centennial Cone", 'Colorado', Big_X)

The scaled features of the original trail are:[[ 2.12173717 -2.00664051  0.04988084  1.53527546  2.6075075  -2.6075075 ]]
The sorted cosine similarity measurements are:[ 1.          0.99983597  0.9997793  ..., -0.99667161 -0.9966879
 -0.99682025]
The trail most similiar to the original trail has features: [ 2.16292354 -2.07733332  0.04988084  1.61729795  2.6075075  -2.6075075 ]
The trail most similar to the original trail is: Golden Gate to Tennessee Beach Loop


,name,location,difficulty,length,ascent,descent,stars,latitude,longitude,summary,...,type_Trail,city/town,state,length_range,ascent_scaled,descent_scaled,difficulty_encoded_scaled,length_scaled,type_Featured Ride_scaled,type_Trail_scaled
original,Centennial Cone,"Idaho Springs, Colorado",Blue,17.2,2405,-2409,4.0,39.7372,-105.3710,One of the better beginner/intermediate rides ...,...,0,Idaho Springs,Colorado,15-20,2.121737,-2.006641,0.049881,1.535275,2.607508,-2.607508
7331,Golden Gate to Tennessee Beach Loop,"Sausalito, California",Blue,17.9,2443,-2477,4.2,37.8330,-122.4829,"Mountain biking near the ""City by the Bay"" - w...",...,0,Sausalito,California,15-20,2.162924,-2.077333,0.049881,1.617298,2.607508,-2.607508
10069,Jackson Demo Forest,"Fort Bragg, California",Blue,16.7,2469,-2471,4.3,39.3355,-123.7566,Redwood forest with fun a singletrack ride!,...,0,Fort Bragg,California,15-20,2.191104,-2.071096,0.049881,1.476688,2.607508,-2.607508
11944,Corner Canyon Front and Back,"Draper, Utah",Blue,16.4,2333,-2333,5.0,40.5060,-111.8455,A 16-mile loop from the Equestrian Center.,...,0,Draper,Utah,15-20,2.043700,-1.927631,0.049881,1.441535,2.607508,-2.607508
15575,Game Creek Loop,"Jackson, Wyoming",Blue,18.5,2435,-2435,4.2,43.4695,-110.7722,A great loop that you can take either way from...,...,0,Jackson,Wyoming,15-20,2.154253,-2.033670,0.049881,1.687603,2.607508,-2.607508
2743,Lower Monroe Truck Trail,"Glendora, California",Blue,16.1,2319,-2319,4.1,34.1562,-117.8367,A great doubletrack to the top of Glendora Mou...,...,0,Glendora,California,15-20,2.028526,-1.913076,0.049881,1.406383,2.607508,-2.607508
7330,Mount Tamalpais Summit Loop,"Mill Valley, California",Blue,16.2,2469,-2468,4.0,37.9186,-122.5538,"""Mount Tam"", the heart of Marin County, is con...",...,0,Mill Valley,California,15-20,2.191104,-2.067977,0.049881,1.418100,2.607508,-2.607508
6531,Monte Bello-Stevens Creek Loop,"Cupertino, California",Blue,17.3,2536,-2538,3.7,37.2945,-122.0819,Spectacular views of Silicon Valley to the Pac...,...,0,Cupertino,California,15-20,2.263722,-2.140749,0.049881,1.546993,2.607508,-2.607508
6104,Shaver Lake Loop,"Auberry, California",Blue,18.4,2355,-2355,3.3,37.1074,-119.3179,Several trails linked to complete the loop. No...,...,0,Auberry,California,15-20,2.067545,-1.950502,0.049881,1.675885,2.607508,-2.607508
16117,Sanford and Son,"Boise, Idaho",Blue,18.4,2521,-2518,4.2,43.6390,-116.1944,See the best of both Hulls Gulch and Military ...,...,0,Boise,Idaho,15-20,2.247464,-2.119957,0.049881,1.675885,2.607508,-2.607508


In [473]:
euclidean_dist_recommendations("Centennial Cone", 'Colorado', Big_X)

The scaled features of the original trail are:[[ 2.12173717 -2.00664051  0.04988084  1.53527546  2.6075075  -2.6075075 ]]
The sorted euclidean distance measurements are:[  0.           0.11134936   0.11585111 ...,  40.45073492  41.03277894
  51.21569953]
The trail most similiar to the original trail has features: [ 2.19110368 -2.07109572  0.04988084  1.47668797  2.6075075  -2.6075075 ]
The trail most similar to the original trail is: Jackson Demo Forest


,name,location,difficulty,length,ascent,descent,stars,latitude,longitude,summary,...,type_Trail,city/town,state,length_range,ascent_scaled,descent_scaled,difficulty_encoded_scaled,length_scaled,type_Featured Ride_scaled,type_Trail_scaled
original,Centennial Cone,"Idaho Springs, Colorado",Blue,17.2,2405,-2409,4.0,39.7372,-105.3710,One of the better beginner/intermediate rides ...,...,0,Idaho Springs,Colorado,15-20,2.121737,-2.006641,0.049881,1.535275,2.607508,-2.607508
10069,Jackson Demo Forest,"Fort Bragg, California",Blue,16.7,2469,-2471,4.3,39.3355,-123.7566,Redwood forest with fun a singletrack ride!,...,0,Fort Bragg,California,15-20,2.191104,-2.071096,0.049881,1.476688,2.607508,-2.607508
7331,Golden Gate to Tennessee Beach Loop,"Sausalito, California",Blue,17.9,2443,-2477,4.2,37.8330,-122.4829,"Mountain biking near the ""City by the Bay"" - w...",...,0,Sausalito,California,15-20,2.162924,-2.077333,0.049881,1.617298,2.607508,-2.607508
11944,Corner Canyon Front and Back,"Draper, Utah",Blue,16.4,2333,-2333,5.0,40.5060,-111.8455,A 16-mile loop from the Equestrian Center.,...,0,Draper,Utah,15-20,2.043700,-1.927631,0.049881,1.441535,2.607508,-2.607508
7330,Mount Tamalpais Summit Loop,"Mill Valley, California",Blue,16.2,2469,-2468,4.0,37.9186,-122.5538,"""Mount Tam"", the heart of Marin County, is con...",...,0,Mill Valley,California,15-20,2.191104,-2.067977,0.049881,1.418100,2.607508,-2.607508
15575,Game Creek Loop,"Jackson, Wyoming",Blue,18.5,2435,-2435,4.2,43.4695,-110.7722,A great loop that you can take either way from...,...,0,Jackson,Wyoming,15-20,2.154253,-2.033670,0.049881,1.687603,2.607508,-2.607508
6104,Shaver Lake Loop,"Auberry, California",Blue,18.4,2355,-2355,3.3,37.1074,-119.3179,Several trails linked to complete the loop. No...,...,0,Auberry,California,15-20,2.067545,-1.950502,0.049881,1.675885,2.607508,-2.607508
10335,Hay Park,"El Jebel, Colorado",Blue,18.6,2365,-2364,3.8,39.3036,-107.1246,A taste of high country riding but close to ci...,...,0,El Jebel,Colorado,15-20,2.078383,-1.959859,0.049881,1.699320,2.607508,-2.607508
2743,Lower Monroe Truck Trail,"Glendora, California",Blue,16.1,2319,-2319,4.1,34.1562,-117.8367,A great doubletrack to the top of Glendora Mou...,...,0,Glendora,California,15-20,2.028526,-1.913076,0.049881,1.406383,2.607508,-2.607508
6531,Monte Bello-Stevens Creek Loop,"Cupertino, California",Blue,17.3,2536,-2538,3.7,37.2945,-122.0819,Spectacular views of Silicon Valley to the Pac...,...,0,Cupertino,California,15-20,2.263722,-2.140749,0.049881,1.546993,2.607508,-2.607508


In [474]:
cos_sim_recommendations("Mustang Trail", 'Colorado', Big_X)

The scaled features of the original trail are:[[-0.30609059 -0.33392379  0.83473587 -0.22234932 -0.38350801  0.38350801]]
The sorted cosine similarity measurements are:[ 1.          0.99944047  0.99754168 ..., -0.76685046 -0.77808136
 -0.81450566]
The trail most similiar to the original trail has features: [-0.32126451 -0.32768619  0.83473587 -0.25750181 -0.38350801  0.38350801]
The trail most similar to the original trail is: Barlow Road


,name,location,difficulty,length,ascent,descent,stars,latitude,longitude,summary,...,type_Trail,city/town,state,length_range,ascent_scaled,descent_scaled,difficulty_encoded_scaled,length_scaled,type_Featured Ride_scaled,type_Trail_scaled
original,Mustang Trail,"Golden, Colorado",Blue/Black,2.2,165,-800,3.9,39.8159,-105.2926,Some of the best rapid fire technical descendi...,...,1,Golden,Colorado,0-5,-0.306091,-0.333924,0.834736,-0.222349,-0.383508,0.383508
6011,Barlow Road,"Almaden Valley, California",Blue/Black,1.9,151,-794,4.0,37.1572,-121.8811,A short but challenging connector trail.,...,1,Almaden Valley,California,0-5,-0.321265,-0.327686,0.834736,-0.257502,-0.383508,0.383508
4619,Saddleback Trail #232,"Santa Fe, New Mexico",Blue/Black,2.5,232,-804,4.3,35.7427,-105.8581,Tight singletrack in a scrub oak forest that r...,...,1,Santa Fe,New Mexico,0-5,-0.233473,-0.338082,0.834736,-0.187197,-0.383508,0.383508
6207,Little Purgatory Trail,"Leeds, Utah",Blue/Black,2.6,103,-849,4.0,37.2273,-113.3581,A descending trail which switchbacks down to t...,...,1,Leeds,Utah,0-5,-0.373289,-0.384864,0.834736,-0.175479,-0.383508,0.383508
7074,Glenwood Horse Trail (3004E) - Onion Mountain,"Buchanan, Virginia",Blue/Black,2.1,244,-839,5.0,37.5073,-79.5132,A sweet singletrack descent off of Onion Mount...,...,1,Buchanan,Virginia,0-5,-0.220466,-0.374468,0.834736,-0.234067,-0.383508,0.383508
5110,Water Canyon Trail,"Los Alamos, New Mexico",Blue/Black,2.5,167,-710,4.5,35.8447,-106.3869,This is a great descent down water canyon to t...,...,1,Los Alamos,New Mexico,0-5,-0.303923,-0.240360,0.834736,-0.187197,-0.383508,0.383508
8986,Tahoe Mountain Trail,"South Lake Tahoe, California",Blue/Black,2.6,89,-773,4.3,38.9053,-120.0270,This trail has beautiful views of the meadows ...,...,1,South Lake Tahoe,California,0-5,-0.388463,-0.305855,0.834736,-0.175479,-0.383508,0.383508
12077,Old Trench Road #181,"Midway, Utah",Blue/Black,2.1,77,-861,3.0,40.5016,-111.5881,This trail has great views at the top and conn...,...,1,Midway,Utah,0-5,-0.401470,-0.397339,0.834736,-0.234067,-0.383508,0.383508
4252,Live Wire 2,"Chattanooga, Tennessee",Blue/Black,2.9,192,-741,4.3,35.0624,-85.3853,A flowing downhill singletrack with some moder...,...,1,Chattanooga,Tennessee,0-5,-0.276827,-0.272587,0.834736,-0.140327,-0.383508,0.383508
13762,Bear Hollow Trail,"Richmond, Utah",Blue/Black,2.6,187,-887,3.5,41.8835,-111.6364,A section of singletrack leading to the Twin C...,...,1,Richmond,Utah,0-5,-0.282246,-0.424369,0.834736,-0.175479,-0.383508,0.383508


In [475]:
euclidean_dist_recommendations("Mustang Trail", 'Colorado', Big_X)

The scaled features of the original trail are:[[-0.30609059 -0.33392379  0.83473587 -0.22234932 -0.38350801  0.38350801]]
The sorted euclidean distance measurements are:[  0.00000000e+00   3.87924421e-02   8.07859718e-02 ...,   4.35908980e+01
   4.41850070e+01   5.47402867e+01]
The trail most similiar to the original trail has features: [-0.32126451 -0.32768619  0.83473587 -0.25750181 -0.38350801  0.38350801]
The trail most similar to the original trail is: Barlow Road


,name,location,difficulty,length,ascent,descent,stars,latitude,longitude,summary,...,type_Trail,city/town,state,length_range,ascent_scaled,descent_scaled,difficulty_encoded_scaled,length_scaled,type_Featured Ride_scaled,type_Trail_scaled
original,Mustang Trail,"Golden, Colorado",Blue/Black,2.2,165,-800,3.9,39.8159,-105.2926,Some of the best rapid fire technical descendi...,...,1,Golden,Colorado,0-5,-0.306091,-0.333924,0.834736,-0.222349,-0.383508,0.383508
6011,Barlow Road,"Almaden Valley, California",Blue/Black,1.9,151,-794,4.0,37.1572,-121.8811,A short but challenging connector trail.,...,1,Almaden Valley,California,0-5,-0.321265,-0.327686,0.834736,-0.257502,-0.383508,0.383508
4619,Saddleback Trail #232,"Santa Fe, New Mexico",Blue/Black,2.5,232,-804,4.3,35.7427,-105.8581,Tight singletrack in a scrub oak forest that r...,...,1,Santa Fe,New Mexico,0-5,-0.233473,-0.338082,0.834736,-0.187197,-0.383508,0.383508
7074,Glenwood Horse Trail (3004E) - Onion Mountain,"Buchanan, Virginia",Blue/Black,2.1,244,-839,5.0,37.5073,-79.5132,A sweet singletrack descent off of Onion Mount...,...,1,Buchanan,Virginia,0-5,-0.220466,-0.374468,0.834736,-0.234067,-0.383508,0.383508
6207,Little Purgatory Trail,"Leeds, Utah",Blue/Black,2.6,103,-849,4.0,37.2273,-113.3581,A descending trail which switchbacks down to t...,...,1,Leeds,Utah,0-5,-0.373289,-0.384864,0.834736,-0.175479,-0.383508,0.383508
8986,Tahoe Mountain Trail,"South Lake Tahoe, California",Blue/Black,2.6,89,-773,4.3,38.9053,-120.0270,This trail has beautiful views of the meadows ...,...,1,South Lake Tahoe,California,0-5,-0.388463,-0.305855,0.834736,-0.175479,-0.383508,0.383508
5110,Water Canyon Trail,"Los Alamos, New Mexico",Blue/Black,2.5,167,-710,4.5,35.8447,-106.3869,This is a great descent down water canyon to t...,...,1,Los Alamos,New Mexico,0-5,-0.303923,-0.240360,0.834736,-0.187197,-0.383508,0.383508
13762,Bear Hollow Trail,"Richmond, Utah",Blue/Black,2.6,187,-887,3.5,41.8835,-111.6364,A section of singletrack leading to the Twin C...,...,1,Richmond,Utah,0-5,-0.282246,-0.424369,0.834736,-0.175479,-0.383508,0.383508
4252,Live Wire 2,"Chattanooga, Tennessee",Blue/Black,2.9,192,-741,4.3,35.0624,-85.3853,A flowing downhill singletrack with some moder...,...,1,Chattanooga,Tennessee,0-5,-0.276827,-0.272587,0.834736,-0.140327,-0.383508,0.383508
16368,Elk Flat Relay,"Driggs, Idaho",Blue/Black,3.0,172,-738,4.0,43.7031,-111.3411,A gorgeous ridgeline trail traversing through ...,...,1,Driggs,Idaho,0-5,-0.298504,-0.269469,0.834736,-0.128609,-0.383508,0.383508


In [484]:
cos_sim_recommendations("North Table Loop", 'Colorado', Big_X)

The scaled features of the original trail are:[[ 0.47211492 -0.47323022  0.04988084  0.3986781   2.6075075  -2.6075075 ]]
The sorted cosine similarity measurements are:[ 1.          0.99992994  0.99986197 ..., -0.99261592 -0.9930131
 -0.99404606]
The trail most similiar to the original trail has features: [ 0.48295343 -0.43684421  0.04988084  0.4221131   2.6075075  -2.6075075 ]
The trail most similar to the original trail is: Triple C to Step Creek Double Loop


,name,location,difficulty,length,ascent,descent,stars,latitude,longitude,summary,...,type_Trail,city/town,state,length_range,ascent_scaled,descent_scaled,difficulty_encoded_scaled,length_scaled,type_Featured Ride_scaled,type_Trail_scaled
original,North Table Loop,"Golden, Colorado",Blue,7.5,883,-934,3.7,39.7812,-105.2301,Big climbs. Rowdy descents. Amazing 360-degree...,...,0,Golden,Colorado,5-10,0.472115,-0.473230,0.049881,0.398678,2.607508,-2.607508
18507,Triple C to Step Creek Double Loop,"Banks, Oregon",Blue,7.7,893,-899,4.0,45.7070,-123.3375,"Combine two short, closeby loops into a nice, ...",...,0,Banks,Oregon,5-10,0.482953,-0.436844,0.049881,0.422113,2.607508,-2.607508
9272,Strand Hill,"Crested Butte, Colorado",Blue,7.5,874,-874,4.5,38.8644,-106.9130,One of the best rides in the valley.,...,0,Crested Butte,Colorado,5-10,0.462360,-0.410854,0.049881,0.398678,2.607508,-2.607508
9829,Watters Smith State Park,"Stonewood, West Virginia",Blue,7.6,876,-873,4.0,39.1719,-80.4074,A great trail system sampling the foothills an...,...,0,Stonewood,West Virginia,5-10,0.464528,-0.409815,0.049881,0.410396,2.607508,-2.607508
20566,Hillside Park,"Anchorage, Alaska",Blue,7.1,887,-887,4.3,61.1386,-149.7467,A great singletrack tour of east Anchorage wit...,...,0,Anchorage,Alaska,5-10,0.476450,-0.424369,0.049881,0.351808,2.607508,-2.607508
6142,Precipice/Sidewinder/Barrel Roll loop,"Santa Clara, Utah",Blue,7.9,946,-952,4.2,37.1291,-113.6632,A fun loop on trails built just for mountain b...,...,0,Santa Clara,Utah,5-10,0.540398,-0.491943,0.049881,0.445548,2.607508,-2.607508
6154,City Creek to Rusty Cliffs Trail,"Saint George, Utah",Blue,8.2,895,-900,2.4,37.1259,-113.5827,A nice ride from the Red Cliffs north of St. G...,...,0,Saint George,Utah,5-10,0.485121,-0.437884,0.049881,0.480701,2.607508,-2.607508
9273,Snodgrass,"Crested Butte, Colorado",Blue,7.3,855,-853,4.2,38.9228,-107.0046,A short but enjoyable singletrack cruise close...,...,0,Crested Butte,Colorado,5-10,0.441767,-0.389023,0.049881,0.375243,2.607508,-2.607508
2937,Hanna Flats Loop,"Big Bear Lake, California",Blue,8.1,940,-949,3.8,34.2699,-116.9477,A local favorite and a must-do loop if you're ...,...,0,Big Bear Lake,California,5-10,0.533894,-0.488824,0.049881,0.468983,2.607508,-2.607508
9371,Red Rock Canyon Tour,"Manitou Springs, Colorado",Blue,7.2,963,-954,4.1,38.8532,-104.8788,Better than Garden Of the Gods!,...,0,Manitou Springs,Colorado,5-10,0.558823,-0.494022,0.049881,0.363526,2.607508,-2.607508


In [485]:
euclidean_dist_recommendations("North Table Loop", 'Colorado', Big_X)

The scaled features of the original trail are:[[ 0.47211492 -0.47323022  0.04988084  0.3986781   2.6075075  -2.6075075 ]]
The sorted euclidean distance measurements are:[  0.00000000e+00   4.46162978e-02   6.31341456e-02 ...,   4.28970102e+01
   4.35083697e+01   5.36947588e+01]
The trail most similiar to the original trail has features: [ 0.48295343 -0.43684421  0.04988084  0.4221131   2.6075075  -2.6075075 ]
The trail most similar to the original trail is: Triple C to Step Creek Double Loop


,name,location,difficulty,length,ascent,descent,stars,latitude,longitude,summary,...,type_Trail,city/town,state,length_range,ascent_scaled,descent_scaled,difficulty_encoded_scaled,length_scaled,type_Featured Ride_scaled,type_Trail_scaled
original,North Table Loop,"Golden, Colorado",Blue,7.5,883,-934,3.7,39.7812,-105.2301,Big climbs. Rowdy descents. Amazing 360-degree...,...,0,Golden,Colorado,5-10,0.472115,-0.473230,0.049881,0.398678,2.607508,-2.607508
18507,Triple C to Step Creek Double Loop,"Banks, Oregon",Blue,7.7,893,-899,4.0,45.7070,-123.3375,"Combine two short, closeby loops into a nice, ...",...,0,Banks,Oregon,5-10,0.482953,-0.436844,0.049881,0.422113,2.607508,-2.607508
9272,Strand Hill,"Crested Butte, Colorado",Blue,7.5,874,-874,4.5,38.8644,-106.9130,One of the best rides in the valley.,...,0,Crested Butte,Colorado,5-10,0.462360,-0.410854,0.049881,0.398678,2.607508,-2.607508
9829,Watters Smith State Park,"Stonewood, West Virginia",Blue,7.6,876,-873,4.0,39.1719,-80.4074,A great trail system sampling the foothills an...,...,0,Stonewood,West Virginia,5-10,0.464528,-0.409815,0.049881,0.410396,2.607508,-2.607508
20566,Hillside Park,"Anchorage, Alaska",Blue,7.1,887,-887,4.3,61.1386,-149.7467,A great singletrack tour of east Anchorage wit...,...,0,Anchorage,Alaska,5-10,0.476450,-0.424369,0.049881,0.351808,2.607508,-2.607508
6142,Precipice/Sidewinder/Barrel Roll loop,"Santa Clara, Utah",Blue,7.9,946,-952,4.2,37.1291,-113.6632,A fun loop on trails built just for mountain b...,...,0,Santa Clara,Utah,5-10,0.540398,-0.491943,0.049881,0.445548,2.607508,-2.607508
6154,City Creek to Rusty Cliffs Trail,"Saint George, Utah",Blue,8.2,895,-900,2.4,37.1259,-113.5827,A nice ride from the Red Cliffs north of St. G...,...,0,Saint George,Utah,5-10,0.485121,-0.437884,0.049881,0.480701,2.607508,-2.607508
9273,Snodgrass,"Crested Butte, Colorado",Blue,7.3,855,-853,4.2,38.9228,-107.0046,A short but enjoyable singletrack cruise close...,...,0,Crested Butte,Colorado,5-10,0.441767,-0.389023,0.049881,0.375243,2.607508,-2.607508
2937,Hanna Flats Loop,"Big Bear Lake, California",Blue,8.1,940,-949,3.8,34.2699,-116.9477,A local favorite and a must-do loop if you're ...,...,0,Big Bear Lake,California,5-10,0.533894,-0.488824,0.049881,0.468983,2.607508,-2.607508
9371,Red Rock Canyon Tour,"Manitou Springs, Colorado",Blue,7.2,963,-954,4.1,38.8532,-104.8788,Better than Garden Of the Gods!,...,0,Manitou Springs,Colorado,5-10,0.558823,-0.494022,0.049881,0.363526,2.607508,-2.607508
